last

In [12]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from math import pi
import signal
import platform
import random  # Added for random message selection

# 사용자 정의 JSON 인코더 (NumPy 데이터 처리용)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int32, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

# 시각화 설정 (한글 폰트 및 음수 표시 설정)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드 및 전처리 (기존 코드 유지)
df = pd.read_csv("건강데이터_2022_2023_합본.csv")
df = df[(df['HE_glu'] >= 50) & (df['HE_glu'] <= 400)]

df['BE3_31'] = df['BE3_31'].astype(str).str.strip().replace({
    '1.0': 0, '2.0': 1, '3.0': 2, '4.0': 3, '5.0': 4, '6.0': 5,
    '7.0': 6, '8.0': 7, '88.0': 0, '99.0': np.nan, 'nan': np.nan
})
df['BE3_31'] = pd.to_numeric(df['BE3_31'], errors='coerce')

df['L_BR_FQ'] = df['L_BR_FQ'].astype(str).str.strip().replace({
    '1.0': 6, '2.0': 3.5, '3.0': 1.5, '4.0': 0, '9.0': np.nan, 'nan': np.nan
})
df['L_BR_FQ'] = pd.to_numeric(df['L_BR_FQ'], errors='coerce')

df['BS3_2'] = df['BS3_2'].replace(888, 0)
df['BS12_47_1'] = df['BS12_47_1'].replace({888: 0, 999: np.nan})
df['BS3_2'] = pd.to_numeric(df['BS3_2'], errors='coerce')
df['BS12_47_1'] = pd.to_numeric(df['BS12_47_1'], errors='coerce')
df['tobacco'] = df[['BS3_2', 'BS12_47_1']].sum(axis=1, skipna=True)
df['tobacco'] = df['tobacco'].astype(str).str.strip().replace({'999.0': np.nan})
df['tobacco'] = pd.to_numeric(df['tobacco'], errors='coerce')

df['BD1_11'] = df['BD1_11'].astype(str).str.strip().replace({
    '1.0': 0, '2.0': 6, '3.0': 12, '4.0': 42, '5.0': 130, '6.0': 286,
    '8.0': 0, '9.0': np.nan, 'nan': np.nan
})
df['BD1_11'] = pd.to_numeric(df['BD1_11'], errors='coerce')

def determine_he_hp2(sbp, dbp):
    if pd.isna(sbp) or pd.isna(dbp):
        return np.nan
    if sbp >= 140 or dbp >= 90:
        return 4
    elif 130 <= sbp <= 139 or 80 <= dbp <= 89:
        return 3
    elif 120 <= sbp <= 129 and dbp < 80:
        return 2
    else:
        return 1

def determine_he_dm_hba1c2(glu):
    if pd.isna(glu):
        return np.nan
    if glu <= 99:
        return 1
    elif 100 <= glu <= 125:
        return 2
    else:
        return 3

def determine_he_obe2(bmi):
    if pd.isna(bmi):
        return np.nan
    if bmi <= 18.5:
        return 1
    elif 18.5 < bmi <= 22.9:
        return 2
    elif 22.9 < bmi <= 24.9:
        return 3
    elif 24.9 < bmi <= 29.9:
        return 4
    elif 29.9 < bmi <= 34.9:
        return 5
    else:
        return 6

df['HE_HP2'] = df.apply(lambda row: determine_he_hp2(row['HE_sbp1'], row['HE_dbp1']), axis=1)
df['HE_DM_HbA1c2'] = df.apply(lambda row: determine_he_dm_hba1c2(row['HE_glu']), axis=1)
df['HE_obe2'] = df.apply(lambda row: determine_he_obe2(row['HE_BMI']), axis=1)
df['HE_HP2'] = pd.to_numeric(df['HE_HP2'], errors='coerce')
df['HE_DM_HbA1c2'] = pd.to_numeric(df['HE_DM_HbA1c2'], errors='coerce')
df['HE_obe2'] = pd.to_numeric(df['HE_obe2'], errors='coerce')

df_clustering = df.dropna(subset=['HE_HP2', 'HE_DM_HbA1c2', 'HE_obe2'])
selected_cols = ['BD1_11', 'tobacco', 'BE3_31', 'L_BR_FQ']

for col in selected_cols:
    if df_clustering[col].isnull().any():
        mode_value = df_clustering[col].mode(dropna=True)
        if not mode_value.empty:
            df_clustering.loc[:, col] = df_clustering[col].fillna(mode_value[0])
        else:
            df_clustering.loc[:, col] = df_clustering[col].fillna(0)

# precalculated_means 및 cluster_names (기존 데이터 유지)
precalculated_means = {
    'HE_HP2': {
        1: pd.DataFrame({
            '1주일 간 음주 빈도': [0.212965, 0.346154, 0.263829, 0.283081, 3.773973, 0.532374, 3.081522],
            '하루 평균 흡연량': [0.180919, 0.803408, 0.243425, 0.696676, 18.815068, 18.654676, 1.434783],
            '1주일 간 걷기 일수': [1.006360, 5.964613, 6.167879, 1.013850, 3.102740, 3.805755, 4.766304],
            '1주일 간 아침식사 빈도': [5.547703, 0.613368, 5.585898, 0.623269, 2.616438, 3.769784, 3.337862]
        }, index=[1, 2, 3, 4, 5, 6, 7]),
        2: pd.DataFrame({
            '1주일 간 음주 빈도': [3.257282, 0.600634, 0.173377, 0.227414, 0.761134, 5.500000, 1.003752],
            '하루 평균 흡연량': [0.582524, 0.444444, 0.102253, 0.382550, 23.385965, 6.716981, 13.719512],
            '1주일 간 걷기 일수': [5.291262, 3.584229, 6.287695, 1.201342, 1.666667, 2.660377, 6.024390],
            '1주일 간 아침식사 빈도': [5.432039, 0.605735, 5.757366, 5.692394, 3.789474, 3.924528, 3.152439]
        }, index=[1, 2, 3, 4, 5, 6, 7]),
        3: pd.DataFrame({
            '1주일 간 음주 빈도': [0.517830, 0.747283, 2.083208, 0.473159, 5.500000, 1.960664],
            '하루 평균 흡연량': [0.255906, 1.091078, 19.352941, 0.420361, 2.595745, 17.242424],
            '1주일 간 걷기 일수': [6.213583, 3.780669, 1.895425, 1.308703, 4.085106, 5.469697],
            '1주일 간 아침식사 빈도': [5.778543, 0.697026, 2.271242, 5.749589, 4.049645, 5.295455]
        }, index=[1, 2, 3, 4, 5, 6]),
        4: pd.DataFrame({
            '1주일 간 음주 빈도': [5.500000, 0.463656, 0.471816, 2.740812, 0.930769],
            '하루 평균 흡연량': [2.284672, 0.332907, 0.317814, 20.272222, 1.310000],
            '1주일 간 걷기 일수': [3.729927, 6.289373, 1.212551, 3.850000, 3.813333],
            '1주일 간 아침식사 빈도': [4.463504, 5.839949, 5.731781, 3.738889, 0.693333]
        }, index=[1, 2, 3, 4, 5]),
    },
    'HE_DM_HbA1c2': {
        1: pd.DataFrame({
            '1주일 간 음주 빈도': [0.679037, 3.699045, 0.205307, 0.495192, 1.637577, 0.247029],
            '하루 평균 흡연량': [0.883784, 1.920382, 0.305901, 0.764344, 19.815618, 0.317173],
            '1주일 간 걷기 일수': [6.015315, 4.474522, 1.086957, 1.079918, 3.561822, 6.187346],
            '1주일 간 아침식사 빈도': [0.591892, 4.639331, 5.596273, 0.627049, 3.206074, 5.622021]
        }, index=[1, 2, 3, 4, 5, 6]),
        2: pd.DataFrame({
            '1주일 간 음주 빈도': [0.467910, 0.868637, 1.435988, 5.500000, 0.474998],
            '하루 평균 흡연량': [0.438253, 1.461412, 19.321555, 5.978166, 0.407047],
            '1주일 간 걷기 일수': [6.262048, 3.768473, 3.653710, 3.890830, 1.222357],
            '1주일 간 아침식사 빈도': [5.800452, 0.623974, 3.793286, 4.109170, 5.717497]
        }, index=[1, 2, 3, 4, 5]),
        3: pd.DataFrame({
            '1주일 간 음주 빈도': [0.365672, 0.374903, 1.797498, 0.454709, 2.443396, 4.320225],
            '하루 평균 흡연량': [0.492537, 0.199495, 21.493976, 0.953271, 19.056604, 1.000000],
            '1주일 간 걷기 일수': [1.223881, 6.353535, 3.795181, 3.943925, 3.622642, 3.662921],
            '1주일 간 아침식사 빈도': [5.763682, 5.886364, 5.879518, 0.668224, 1.179245, 5.382022]
        }, index=[1, 2, 3, 4, 5, 6]),
    },
    'HE_obe2': {
        1: pd.DataFrame({
            '1주일 간 음주 빈도': [3.462264, 0.056121, 0.347728, 0.147807, 1.490074, 0.463646],
            '하루 평균 흡연량': [3.226415, 0.073930, 0.485380, 0.297521, 19.774194, 0.698630],
            '1주일 간 걷기 일수': [3.094340, 0.443580, 6.187135, 0.719008, 3.967742, 5.945205],
            '1주일 간 아침식사 빈도': [2.943396, 5.659533, 5.576023, 0.619835, 3.064516, 0.534247]
        }, index=[1, 2, 3, 4, 5, 6]),
        2: pd.DataFrame({
            '1주일 간 음주 빈도': [1.346814, 0.619275, 0.418860, 5.237327, 0.692170, 0.278670],
            '하루 평균 흡연량': [18.386266, 0.748092, 0.245912, 7.737327, 0.670251, 0.306064],
            '1주일 간 걷기 일수': [3.497854, 1.219466, 6.217444, 3.718894, 6.069892, 1.216554],
            '1주일 간 아침식사 빈도': [3.343348, 0.885496, 5.662326, 4.158986, 0.585125, 5.677575]
        }, index=[1, 2, 3, 4, 5, 6]),
        3: pd.DataFrame({
            '1주일 간 음주 빈도': [0.428221, 0.465444, 1.405400, 0.733680, 5.500000],
            '하루 평균 흡연량': [0.353503, 0.406190, 19.929293, 0.981982, 5.000000],
            '1주일 간 걷기 일수': [1.168790, 6.233075, 3.691919, 3.645045, 4.143836],
            '1주일 간 아침식사 빈도': [5.685510, 5.787234, 3.845960, 0.740541, 4.191781]
        }, index=[1, 2, 3, 4, 5]),
        4: pd.DataFrame({
            '1주일 간 음주 빈도': [1.745897, 0.476494, 0.474684, 4.074257, 0.400403],
            '하루 평균 흡연량': [20.120000, 0.413905, 0.444304, 2.267327, 0.994178],
            '1주일 간 걷기 일수': [3.673333, 6.226354, 1.289873, 4.603960, 3.608443],
            '1주일 간 아침식사 빈도': [3.465000, 5.767583, 5.724684, 2.785479, 0.788937]
        }, index=[1, 2, 3, 4, 5]),
        5: pd.DataFrame({
            '1주일 간 음주 빈도': [0.313474, 0.188658, 1.018219, 4.313953, 0.697115, 2.906780, 0.266827],
            '하루 평균 흡연량': [0.215686, 0.218978, 17.921053, 3.069767, 23.687500, 2.000000, 1.331250],
            '1주일 간 걷기 일수': [6.202614, 1.160584, 3.368421, 2.488372, 3.781250, 4.915254, 3.893750],
            '1주일 간 아침식사 빈도': [5.722222, 5.635036, 5.605263, 5.174419, 0.984375, 0.838983, 0.659375]
        }, index=[1, 2, 3, 4, 5, 6, 7]),
        6: pd.DataFrame({
            '1주일 간 음주 빈도': [0.202830, 5.500000, 0.407692, 0.332308, 1.230769],
            '하루 평균 흡연량': [0.198113, 7.692308, 0.642857, 0.400000, 20.360000],
            '1주일 간 걷기 일수': [0.424528, 5.230769, 3.942857, 6.480000, 3.840000],
            '1주일 간 아침식사 빈도': [5.787736, 4.730769, 0.600000, 5.625000, 3.480000]
        }, index=[1, 2, 3, 4, 5])
    }
}

cluster_names = {
    'HE_HP2': {
        1: {
            1: "블러드노멀 게으름뱅이",
            2: "블러드노멀 편식러",
            3: "블러드노멀 워커",
            4: "블러드노멀 게으름뱅이",
            5: "블러드노멀 스모키",
            6: "블러드노멀 스모키",
            7: "블러드노멀 루틴러"
        },
        2: {
            1: "블러드어웨어 모닝요정",
            2: "블러드어웨어 편식러",
            3: "블러드어웨어 워커",
            4: "블러드어웨어 게으름뱅이",
            5: "블러드어웨어 스모키",
            6: "블러드어웨어 게으름뱅이",
            7: "블러드어웨어 스모키"
        },
        3: {
            1: "블러드프리 워커",
            2: "블러드프리 편식러",
            3: "블러드프리 스모키",
            4: "블러드프리 게으름뱅이",
            5: "블러드프리 루틴러",
            6: "블러드프리 스모키"
        },
        4: {
            1: "블러드하이 루틴러",
            2: "블러드하이 워커",
            3: "블러드하이 게으름뱅이",
            4: "블러드하이 스모키",
            5: "블러드하이 편식러"
        }
    },
    'HE_DM_HbA1c2': {
        1: {
            1: "슈가노멀 워커",
            2: "슈가노멀 루틴러",
            3: "슈가노멀 게으름뱅이",
            4: "슈가노멀 게으름뱅이",
            5: "슈가노멀 스모키",
            6: "슈가노멀 워커"
        },
        2: {
            1: "슈가프리 워커",
            2: "슈가프리 편식러",
            3: "슈가프리 스모키",
            4: "슈가프리 루틴러",
            5: "슈가프리 게으름뱅이"
        },
        3: {
            1: "슈가하이 게으름뱅이",
            2: "슈가하이 워커",
            3: "슈가하이 스모키",
            4: "슈가하이 편식러",
            5: "슈가하이 스모키",
            6: "슈가하이 모닝요정"
        }
    },
    'HE_obe2': {
        1: {
            1: "바디로우 루틴러",
            2: "바디로우 게으름뱅이",
            3: "바디로우 워커",
            4: "바디로우 게으름뱅이",
            5: "바디로우 스모키",
            6: "바디로우 편식러"
        },
        2: {
            1: "바디노멀 스모키",
            2: "바디노멀 게으름뱅이",
            3: "바디노멀 워커",
            4: "바디노멀 루틴러",
            5: "바디노멀 워커",
            6: "바디노멀 게으름뱅이"
        },
        3: {
            1: "바디라이트 게으름뱅이",
            2: "바디라이트 워커",
            3: "바디라이트 스모키",
            4: "바디라이트 편식러",
            5: "바디라이트 루틴러"
        },
        4: {
            1: "바디하이 스모키",
            2: "바디하이 워커",
            3: "바디하이 게으름뱅이",
            4: "바디하이 루틴러",
            5: "바디하이 편식러"
        },
        5: {
            1: "바디폼 워커",
            2: "바디폼 게으름뱅이",
            3: "바디폼 스모키",
            4: "바디폼 게으름뱅이",
            5: "바디폼 스모키",
            6: "바디폼 편식러",
            7: "바디폼 편식러"
        },
        6: {
            1: "바디슈퍼 게으름뱅이",
            2: "바디슈퍼 루틴러",
            3: "바디슈퍼 편식러",
            4: "바디슈퍼 워커",
            5: "바디슈퍼 스모키"
        }
    }
}

def determine_cluster_with_precalculated_means(user_data_weekly_daily, condition_value, scaler, precalculated_means_dict, selected_cols, condition_type):
    condition_value_int = int(condition_value)

    if condition_type not in precalculated_means_dict or condition_value_int not in precalculated_means_dict[condition_type]:
        return None, None

    cluster_means_precalculated_text_names = precalculated_means_dict[condition_type][condition_value_int].copy()

    text_to_internal_col_mapping = {
        '1주일 간 음주 빈도': 'BD1_11',
        '하루 평균 흡연량': 'tobacco',
        '1주일 간 걷기 일수': 'BE3_31',
        '1주일 간 아침식사 빈도': 'L_BR_FQ'
    }

    cluster_means_precalculated_internal_names = pd.DataFrame(index=cluster_means_precalculated_text_names.index)
    for internal_col in selected_cols:
        text_col = next((k for k, v in text_to_internal_col_mapping.items() if v == internal_col), None)
        if text_col and text_col in cluster_means_precalculated_text_names.columns:
            cluster_means_precalculated_internal_names[internal_col] = cluster_means_precalculated_text_names[text_col]
        else:
            cluster_means_precalculated_internal_names[internal_col] = np.nan

    if cluster_means_precalculated_internal_names.isnull().values.any():
        return None, None

    scaled_precalculated_centroids = scaler.transform(cluster_means_precalculated_internal_names[selected_cols])

    # Convert user data into a DataFrame with the same column names as used in fitting the scaler
    user_data_df = pd.DataFrame(
        [user_data_weekly_daily],
        columns=['BD1_11', 'tobacco', 'BE3_31', 'L_BR_FQ']
    )
    user_data_scaled_for_predict = scaler.transform(user_data_df)

    # Dynamically determine the closest cluster based on Euclidean distance
    closest_cluster = np.argmin(np.linalg.norm(scaled_precalculated_centroids - user_data_scaled_for_predict, axis=1)) + 1

    cluster_means = precalculated_means_dict[condition_type][condition_value_int].iloc[closest_cluster - 1]
    return closest_cluster, cluster_means


def save_radar_chart(user_values, cluster_values, chart_labels, user_name, filename, cluster_name):
    def timeout_handler(signum, frame):
        raise TimeoutError("Chart rendering timed out")

    is_windows = platform.system() == "Windows"
    timeout_seconds = 10

    if not is_windows:
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout_seconds)

    try:
        values_user = user_values + user_values[:1]
        values_cluster = cluster_values + cluster_values[:1]
        angles = [n / float(len(chart_labels)) * 2 * pi for n in range(len(chart_labels))]
        angles += angles[:1]

        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
        ax.set_ylim(0, 7)
        ax.fill(angles, values_user, color='red', alpha=0.25, label=user_name)
        ax.plot(angles, values_user, color='red', linewidth=2)
        ax.fill(angles, values_cluster, color='blue', alpha=0.25, label=f"{cluster_name} 평균")
        ax.plot(angles, values_cluster, color='blue', linewidth=2)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(chart_labels, fontsize=10)
        ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
        ax.grid(True)

        fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
        os.makedirs("charts", exist_ok=True)
        chart_path = os.path.join("charts", filename)
        plt.savefig(chart_path, dpi=100)
        plt.close()
        return chart_path

    except TimeoutError:
        plt.close()
        return None
    finally:
        if not is_windows:
            signal.alarm(0)

def calculate_health_score(cluster_mean, condition_type):
    weights = {
        'HE_HP2': {
            'tobacco': 0.383886,
            'BE3_31': 0.277795,
            'BD1_11': 0.188051,
            'L_BR_FQ': 0.150268
        },
        'HE_DM_HbA1c2': {
            'tobacco': 0.362728,
            'BE3_31': 0.275424,
            'BD1_11': 0.199061,
            'L_BR_FQ': 0.162787
        },
        'HE_obe2': {
            'tobacco': 0.398547,
            'BE3_31': 0.265374,
            'BD1_11': 0.198054,
            'L_BR_FQ': 0.138025
        }
    }

    if condition_type not in weights:
        raise ValueError(f"알 수 없는 condition_type: {condition_type}")

    w = weights[condition_type]
    
    score = (
        -cluster_mean['1주일 간 음주 빈도'] * w['BD1_11']
        -cluster_mean['하루 평균 흡연량'] * w['tobacco']
        +cluster_mean['1주일 간 걷기 일수'] * w['BE3_31']
        +cluster_mean['1주일 간 아침식사 빈도'] * w['L_BR_FQ']
    )
    return score

def calculate_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (norm_vec1 * norm_vec2)

def find_healthier_and_similar_cluster(user_data_raw, user_cluster_means, condition_value, precalculated_means_dict, condition_type):
    condition_value_int = int(condition_value)

    if condition_type not in precalculated_means_dict or condition_value_int not in precalculated_means_dict[condition_type]:
        return None, None

    cluster_means_all = precalculated_means_dict[condition_type][condition_value_int]
    health_scores = cluster_means_all.apply(lambda row: calculate_health_score(row, condition_type), axis=1)

    user_cluster_score = calculate_health_score(user_cluster_means, condition_type)

    user_data_vector_raw = np.array([
        user_data_raw[0],  # 음주 빈도
        user_data_raw[1],  # 흡연량
        user_data_raw[2],  # 걷기 일수
        user_data_raw[3]   # 아침식사 빈도
    ])

    healthier_cluster_index = None
    max_cosine_similarity = -1
    for idx in cluster_means_all.index:
        current_cluster_mean = cluster_means_all.loc[idx]
        current_score = calculate_health_score(current_cluster_mean, condition_type)
        if current_score > user_cluster_score:
            cluster_mean_vector_raw = current_cluster_mean[['1주일 간 음주 빈도', '하루 평균 흡연량', '1주일 간 걷기 일수', '1주일 간 아침식사 빈도']].values
            sim = calculate_cosine_similarity(user_data_vector_raw, cluster_mean_vector_raw)
            if sim > max_cosine_similarity:
                max_cosine_similarity = sim
                healthier_cluster_index = idx

    if healthier_cluster_index is not None:
        return healthier_cluster_index, cluster_means_all.loc[healthier_cluster_index]
    return 'self', user_cluster_means

def integrated_health_service(user_data):
    try:
        name = user_data['user_name']
        required_keys = ['HE_sbp1', 'HE_dbp1', 'HE_glu', 'HE_BMI', 'weekly_alcohol', 'daily_smoking', 'weekly_exercise', 'daily_veg']
        for key in required_keys:
            if key not in user_data or user_data[key] is None:
                return {"error": f"{key} 값이 누락되었거나 None입니다."}

        try:
            sbp = float(user_data['HE_sbp1'])
            dbp = float(user_data['HE_dbp1'])
            glu = float(user_data['HE_glu'])
            bmi = float(user_data['HE_BMI'])
            weekly_alcohol = float(user_data['weekly_alcohol'])
            daily_smoking = float(user_data['daily_smoking'])
            weekly_exercise = float(user_data['weekly_exercise'])
            daily_breakfast = float(user_data['daily_veg'])
        except (ValueError, TypeError) as e:
            return {"error": f"입력값 변환 실패: {str(e)}"}

        disease_labels = {
            'HE_HP2': {1: "정상", 2: "주의 혈압", 3: "고혈압 전단계", 4: "고혈압"},
            'HE_DM_HbA1c2': {1: "정상", 2: "당뇨 전단계", 3: "당뇨"},
            'HE_obe2': {1: "저체중", 2: "정상", 3: "과체중", 4: "비만", 5: "고도비만", 6: "초고도비만"},
        }

        disease_display_names = {
            'HE_HP2': "고혈압",
            'HE_DM_HbA1c2': "당뇨",
            'HE_obe2': "비만",
        }

        disease_funcs = {
            'HE_HP2': lambda sbp, dbp: determine_he_hp2(sbp, dbp),
            'HE_DM_HbA1c2': lambda glu, _: determine_he_dm_hba1c2(glu),
            'HE_obe2': lambda bmi, _: determine_he_obe2(bmi),
        }

        json_results = []
        categories = ['1주일 간 음주 빈도', '하루 평균 흡연량', '1주일 간 걷기 일수', '1주일 간 아침식사 빈도']
        chart_labels = ['음주 빈도', '흡연량', '걷기 일수', '아침식사 빈도']
        user_inputs = [weekly_alcohol, daily_smoking, weekly_exercise, daily_breakfast]

        # --- 랜덤 멘트 리스트 정의 시작 ---
        # 음주 빈도 0일 때
        alcohol_zero_messages = [
            "잘하고 있습니다! 꾸준히 건강한 생활을 유지하세요!",
            "음주를 하지 않는 당신, 건강한 습관을 잘 지키고 계시네요!",
            "훌륭합니다! 이대로 금주 습관을 이어가세요!",
            "음주 빈도 0회, 아주 바람직한 생활 습관입니다!",
            "건강을 위한 최고의 선택, 음주를 삼가고 계시군요!"
        ]
        # 음주 빈도 줄여야 할 때
        alcohol_reduce_messages = [
            "음주 빈도는 {abs_diff_alcohol}회 줄여야 합니다. 조금만 노력해 볼까요?",
            "더 건강해지려면 음주를 {abs_diff_alcohol}회 줄이는 것이 필요합니다.",
            "현재 음주량에서 {abs_diff_alcohol}회 정도 줄이는 것을 목표로 해보세요.",
            "음주량을 {abs_diff_alcohol}회 줄이면 더 나은 건강을 기대할 수 있습니다.",
            "건강을 위해 음주 빈도를 {abs_diff_alcohol}회 감소시키는 것을 추천합니다."
        ]
        # 음주 빈도 적정일 때
        alcohol_moderate_messages = [
            "적절한 음주 습관을 유지하고 계십니다! 지금보다 더 건강한 생활을 원하신다면 금주에도 도전해보세요.",
            "음주 빈도는 적정 수준입니다. 건강을 위해 더 나은 변화를 시도해 볼 수도 있어요.",
            "균형 잡힌 음주 습관을 가지고 계시네요! 한 걸음 더 나아가 금주를 고려해보세요.",
            "음주 빈도, 이대로도 좋지만 건강 목표가 있다면 금주도 좋은 방법입니다.",
            "현재 음주량은 적절합니다. 더 큰 건강 이점을 위해 금주를 시도해 보는 건 어떠세요?"
        ]

        # 흡연량 0개비일 때
        smoking_zero_messages = [
            "훌륭합니다! 금연은 건강을 위한 최고의 선물입니다.",
            "흡연량이 0개비! 아주 바람직한 건강 습관을 가지고 계시네요!",
            "담배로부터 자유로운 당신, 건강한 생활을 지속하세요!",
            "금연을 통해 건강을 굳건히 지키고 계십니다. 대단해요!",
            "흡연 0개비, 건강한 폐를 위한 최고의 선택입니다!"
        ]
        # 흡연량 줄여야 할 때
        smoking_reduce_messages = [
            "흡연량은 {abs_diff_smoking}개비 줄여야 합니다. 건강을 위해 금연에 도전해 보세요!",
            "더 건강한 삶을 위해 흡연량을 {abs_diff_smoking}개비 줄이는 것이 중요합니다.",
            "현재 흡연량에서 {abs_diff_smoking}개비 정도 줄이는 것을 목표로 삼아보세요.",
            "흡연량을 {abs_diff_smoking}개비 줄이면 건강 개선에 큰 도움이 될 것입니다.",
            "건강을 위해 흡연량을 {abs_diff_smoking}개비 감소시키는 것을 강력히 추천합니다."
        ]
        # 흡연량 적정일 때 (금연 중 권고에 대한 멘트)
        smoking_moderate_messages = [
            "금연에 성공하셨거나 흡연량이 적정입니다. 건강을 위해 꾸준히 금연을 유지하는 것이 중요합니다!",
            "흡연 습관은 적정 수준입니다. 앞으로도 이 상태를 잘 유지해주세요!",
            "흡연량 관리를 잘하고 계시네요! 완벽한 건강을 위해 금연을 추천합니다.",
            "현재 흡연량은 적정하지만, 건강을 위해 완전한 금연을 고려해보는 건 어떠세요?",
            "적절한 흡연 습관을 유지하고 계십니다. 금연으로 더 건강한 삶을 만들어가세요!"
        ]

        # 걷기 일수 늘려야 할 때
        exercise_increase_messages = [
            "걷기 일수는 {abs_diff_exercise}회 더 늘려야 합니다. 활기찬 하루를 위해 조금 더 걸어볼까요?",
            "건강 증진을 위해 걷기 운동을 {abs_diff_exercise}회 더 늘리는 것을 권장합니다.",
            "목표 걷기 일수에 도달하려면 {abs_diff_exercise}회 더 걷는 것이 필요합니다.",
            "지금보다 {abs_diff_exercise}회 더 걸으면 훨씬 건강해질 수 있습니다.",
            "걷기 일수를 {abs_diff_exercise}회 늘려 건강한 습관을 만들어보세요!"
        ]
        # 걷기 일수 충분할 때
        exercise_sufficient_messages = [
            "걷기 일수는 충분합니다. 현재의 활동량을 유지하세요!",
            "충분히 걷고 계시네요! 아주 좋은 생활 습관입니다.",
            "활동량이 충분하여 건강을 잘 관리하고 계십니다.",
            "매일매일 꾸준히 걷는 당신, 건강을 위한 최고의 노력을 하고 계시네요!",
            "걷기 일수, 이대로도 완벽합니다! 건강한 발걸음을 응원합니다."
        ]
        # 걷기 일수 적정일 때 (충분과 유사하지만 멘트 다양화를 위해)
        exercise_moderate_messages = [
            "걷기 일수는 적정입니다. 꾸준히 건강한 발걸음을 이어가세요!",
            "적당한 걷기 운동을 하고 계시네요. 이대로 잘 유지해 주세요!",
            "걷기 일수는 균형 잡힌 수준입니다. 꾸준함이 중요해요!",
            "활동량이 적절합니다. 지금처럼 활기찬 생활을 계속하세요!",
            "걷기 일수는 적정합니다. 건강한 습관을 잘 지키고 계시네요!"
        ]

        # 아침식사 빈도 늘려야 할 때
        breakfast_increase_messages = [
            "아침식사 빈도는 {abs_diff_breakfast}회 더 늘려야 합니다. 건강한 하루를 위해 아침 식사를 시작해 보세요!",
            "규칙적인 아침 식사를 위해 {abs_diff_breakfast}회 더 챙겨 먹는 것을 추천합니다.",
            "아침 식사를 {abs_diff_breakfast}회 늘려보세요. 하루의 활력이 달라질 거예요!",
            "건강한 식습관을 위해 아침 식사 빈도를 {abs_diff_breakfast}회 늘리는 노력이 필요합니다.",
            "아침식사 횟수를 {abs_diff_breakfast}회 늘려 균형 잡힌 영양을 섭취해 보세요."
        ]
        # 아침식사 빈도 충분할 때
        breakfast_sufficient_messages = [
            "아침식사 빈도는 충분합니다. 규칙적인 식습관을 잘 유지하고 계시네요!",
            "매일 아침을 잘 챙겨 먹는 당신, 건강한 생활 습관을 실천하고 계십니다!",
            "아침 식사량이 충분하여 아주 좋습니다. 이대로 꾸준히 유지하세요!",
            "하루를 건강하게 시작하는 당신의 아침식사 습관, 훌륭합니다!",
            "아침식사 빈수, 이대로도 좋습니다! 건강한 식탁을 응원합니다."
        ]
        # 아침식사 빈도 적정일 때 (충분과 유사하지만 멘트 다양화를 위해)
        breakfast_moderate_messages = [
            "아침식사 빈도는 적정입니다. 꾸준히 건강한 아침을 챙겨 드세요!",
            "적절한 아침식사 습관을 가지고 계시네요. 균형 잡힌 식사를 이어가세요!",
            "아침 식사 빈도는 좋습니다. 건강한 습관을 잘 지키고 계시네요!",
            "하루의 시작을 위한 아침 식사는 적절합니다. 지금처럼 잘 드세요!",
            "아침식사 빈수, 이대로도 좋습니다. 건강한 식습관을 응원합니다!"
        ]
        # --- 랜덤 멘트 리스트 정의 끝 ---

        for disease_code in ['HE_HP2', 'HE_DM_HbA1c2', 'HE_obe2']:
            if disease_code == 'HE_HP2':
                val = disease_funcs[disease_code](sbp, dbp)
            else:
                val = disease_funcs[disease_code](glu if disease_code == 'HE_DM_HbA1c2' else bmi, None)

            if pd.isna(val):
                continue
            val_int = int(val)

            if disease_code not in precalculated_means or val_int not in precalculated_means[disease_code]:
                continue

            df_group = df_clustering[df_clustering[disease_code] == val_int][selected_cols].copy()
            if df_group.empty:
                cluster_means_df = precalculated_means[disease_code][val_int][categories]
                scaler = StandardScaler()
                scaler.fit(cluster_means_df)
            else:
                scaler = StandardScaler()
                scaler.fit(df_group[selected_cols])

            cluster, cluster_means = determine_cluster_with_precalculated_means(user_inputs, val_int, scaler, precalculated_means, selected_cols, disease_code)
            if cluster is None:
                continue

            target_idx, target_means = find_healthier_and_similar_cluster(user_inputs, cluster_means, val_int, precalculated_means, disease_code)
            if target_idx is None:
                target_idx = cluster
                target_means = cluster_means

            # Get cluster name
            cluster_name = cluster_names.get(disease_code, {}).get(val_int, {}).get(cluster, "Unknown Cluster")
            healthier_cluster_name = cluster_names.get(disease_code, {}).get(val_int, {}).get(target_idx, "Unknown Cluster") if target_idx != 'self' else cluster_name

            image_filename = f"{name}_{disease_code}.png"
            chart_path = save_radar_chart(user_inputs, target_means.tolist(), chart_labels, name, image_filename, cluster_name)

            summary_messages = []
            user_values = user_inputs

            # Round comparison_cluster_values to 2 decimal places
            rounded_comparison_cluster_values = {k: round(v, 2) for k, v in dict(zip(categories, target_means.tolist())).items()}

            # --- 음주 빈도 멘트 ---
            if user_values[0] == 0:
                summary_messages.append(random.choice(alcohol_zero_messages))
            else:
                diff_alcohol = round(user_values[0] - target_means['1주일 간 음주 빈도'], 2)
                if diff_alcohol > 0:
                    summary_messages.append(random.choice(alcohol_reduce_messages).format(abs_diff_alcohol=abs(diff_alcohol)))
                elif diff_alcohol < 0:
                    summary_messages.append(random.choice(alcohol_moderate_messages))
                else:
                    summary_messages.append(random.choice(alcohol_moderate_messages))  # 적정 멘트도 랜덤

            # --- 흡연량 멘트 ---
            if user_values[1] == 0:
                summary_messages.append(random.choice(smoking_zero_messages))
            else:
                diff_smoking = round(user_values[1] - target_means['하루 평균 흡연량'], 2)
                if diff_smoking > 0:
                    summary_messages.append(random.choice(smoking_reduce_messages).format(abs_diff_smoking=abs(diff_smoking)))
                elif diff_smoking < 0:
                    summary_messages.append(random.choice(smoking_moderate_messages))
                else:
                    summary_messages.append(random.choice(smoking_moderate_messages))  # 적정 멘트도 랜덤

            # --- 걷기 일수 멘트 ---
            diff_exercise = round(target_means['1주일 간 걷기 일수'] - user_values[2], 2)
            if diff_exercise > 0:
                summary_messages.append(random.choice(exercise_increase_messages).format(abs_diff_exercise=abs(diff_exercise)))
            elif diff_exercise < 0:
                summary_messages.append(random.choice(exercise_sufficient_messages))
            else:
                summary_messages.append(random.choice(exercise_moderate_messages))  # 적정 멘트도 랜덤

            # --- 아침식사 빈도 멘트 ---
            diff_breakfast = round(target_means['1주일 간 아침식사 빈도'] - user_values[3], 2)
            if diff_breakfast > 0:
                summary_messages.append(random.choice(breakfast_increase_messages).format(abs_diff_breakfast=abs(diff_breakfast)))
            elif diff_breakfast < 0:
                summary_messages.append(random.choice(breakfast_sufficient_messages))
            else:
                summary_messages.append(random.choice(breakfast_moderate_messages))  # 적정 멘트도 랜덤

            json_results.append({
                "disease_name": disease_display_names[disease_code],
                "disease_status": disease_labels[disease_code][val_int],
                "cluster_name": cluster_name,
                "healthier_cluster_name": healthier_cluster_name,
                "user_values": dict(zip(categories, user_inputs)),
                "comparison_cluster_values": rounded_comparison_cluster_values,
                "radar_chart_image_path": chart_path,
                "summary_messages": summary_messages
            })

        return {
            "user_name": name,
            "analysis_results": json_results
        }

    except Exception as e:
        return {"error": str(e)}

if __name__ == "__main__":
    sample_user_data = {
        'user_name': '김미정',
        'HE_sbp1': 135,
        'HE_dbp1': 85,
        'HE_glu': 99,
        'HE_BMI': 20,
        'weekly_alcohol': 5,
        'daily_smoking': 5,
        'weekly_exercise': 2,
        'daily_veg': 3
    }

    integrated_output = integrated_health_service(sample_user_data)
    print(json.dumps(integrated_output, indent=2, ensure_ascii=False, cls=NumpyEncoder))

C:\Users\asia\AppData\Local\Temp\ipykernel_19540\2140757531.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['BE3_31'] = df['BE3_31'].astype(str).str.strip().replace({
C:\Users\asia\AppData\Local\Temp\ipykernel_19540\2140757531.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['L_BR_FQ'] = df['L_BR_FQ'].astype(str).str.strip().replace({
C:\Users\asia\AppData\Local\Temp\ipykernel_19540\2140757531.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the

{
  "user_name": "김미정",
  "analysis_results": [
    {
      "disease_name": "고혈압",
      "disease_status": "고혈압 전단계",
      "cluster_name": "블러드프리 루틴러",
      "healthier_cluster_name": "블러드프리 편식러",
      "user_values": {
        "1주일 간 음주 빈도": 5.0,
        "하루 평균 흡연량": 5.0,
        "1주일 간 걷기 일수": 2.0,
        "1주일 간 아침식사 빈도": 3.0
      },
      "comparison_cluster_values": {
        "1주일 간 음주 빈도": 0.75,
        "하루 평균 흡연량": 1.09,
        "1주일 간 걷기 일수": 3.78,
        "1주일 간 아침식사 빈도": 0.7
      },
      "radar_chart_image_path": "charts\\김미정_HE_HP2.png",
      "summary_messages": [
        "더 건강해지려면 음주를 4.25회 줄이는 것이 필요합니다.",
        "더 건강한 삶을 위해 흡연량을 3.91개비 줄이는 것이 중요합니다.",
        "지금보다 1.78회 더 걸으면 훨씬 건강해질 수 있습니다.",
        "하루를 건강하게 시작하는 당신의 아침식사 습관, 훌륭합니다!"
      ]
    },
    {
      "disease_name": "당뇨",
      "disease_status": "정상",
      "cluster_name": "슈가노멀 루틴러",
      "healthier_cluster_name": "슈가노멀 워커",
      "user_values": {
        "1주일 간 음주 빈도": 5.0,
        "하루 평균 흡연량": 5.0,